# Coursera_Capstone - Week 3 Submission

## Problem 1

### Install Required Libraries

In [ ]:
"""
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install geocoder
!pip install wget
"""

### Import Libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Download `index.html` From Wikipedia Using `wget`

In [ ]:
!wget -O index.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Read The `.html` File and Extract the `<table>` From It

In [2]:
fp = open('index.html', encoding="utf-8")
soup = BeautifulSoup(fp, 'html5lib')
table = soup.table.prettify()

### Create a Pandas Dataframe for the `<table>`

In [3]:
df = pd.read_html(table, na_values=[])
df = df[0]

### Process the Dataframe with the Following Requirements:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

#### Rename `Postal code` to `PostalCode`.

In [4]:
df.rename(columns = {'Postal code':'PostalCode'}, inplace = True) 

#### Remove any "Not assigned" from the Borough column.

In [5]:
df = df[df.Borough != 'Not assigned']

#### Search for any "Not assigned" from the Neighborhood Column and replace with its corresponding Borough.

In [6]:
for index, row in df.iterrows():
    if(row[2]=='NaN' or row[2]== '' or row[2]=='Not assigned'):
        print('Found one at index: {}', index)
        df.set_value(index, 'Neighborhood', row[1])
    else:
        continue

#### Merge any duplicate postal codes into one.

**Note: Wikipedia has already merged duplicate zipcodes together and is delimitated by `"/"`. We will therefore replace `"/"` with `","`.**

In [7]:
duplicates = df.duplicated()
duplicates # Shows that there are no duplicate zipcodes

2      False
3      False
4      False
5      False
6      False
       ...  
160    False
165    False
168    False
169    False
178    False
Length: 103, dtype: bool

In [8]:
df.replace({'Neighborhood': r' /'}, {'Neighborhood': ','}, regex=True, inplace=True)

#### Reset the index to cleanup dataframe.

In [9]:
df.reset_index(drop=True, inplace=True)

#### Output the shape of the dataframe

In [10]:
df.shape

(103, 3)

### Final Output

In [11]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


## Problem 2

### Download `Geospatial_Coordinates.csv` From https://cocl.us/Geospatial_data

In [ ]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

### Read the `.csv` file into a Pandas DataFrame.

In [12]:
coord = pd.read_csv('Geospatial_Coordinates.csv')

### Use `geocoder` to obtain latitude and longitude of our postal codes.
**Note: I had an issue with executing the following code on the bottom when testing out a random postal_code from our DataFrame**

In [ ]:
"""
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M1B'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]"""


### Since `geocoder` was unable to execute and was stalled at `Executing cell`,  the `Geospatial_Coordinates.csv` we will be used to merge the coordinates with our DataFrame in lieu of `geocoder`.

#### Rename `Postal Code` to `PostalCode` for matching columns to join.

In [13]:
coord.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 

#### Merge

In [14]:
result = pd.merge(df, coord, how="left", on=['PostalCode'])

### Final Product

In [15]:
result

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
